In [1]:
reset()

In [2]:
#\\\\ some auxiliary functions \\\\#

def print_info():
    print("index=",index)
    print("R=",R)
    print("n=",n)
    print("R_bits",R_bits)
    
def bits(n):
    return floor(log(n,2))+1

def check_indexes(n,R,index,mode,R_bits):
    '''
    we make some checks concerning the constraints on n,R and mode
    ''' 
    if n%2==1:
        print("n must be even")
        return 1,0,0
    if R%mode != 0:
        print("R must be divisible by ",mode)
        return 1,0,0
    for x in R_bits[index-1]:
        if x>1:
            if R%x != 0:
                print("R must be divisible by ",x)
                return 1,0,0
        else:
            continue
    
    return 0,0,0

    
def rank_of_L(A):
    '''
    outputs three integers. 
    [1] the rank of the lattice Ax=0, which is equal to (number of columns of A)-(rank(A)), 
    This is in fact the dimension of the nullspace(A).
    [2] the number of columns k and 
    [3] the rank r of A
    '''
    k=A.dimensions()[1] # number of columns
    r=A.rank()
    return k-r,k,r

#basis of the lattice L:ΑΧ=0
def kernel(A):
    Rank,k,r = rank_of_L(A)
    #print("rank of the lattice:",Rank )
    Q=A.smith_form()[2]
    X=matrix(ZZ,[Q.column(i) for i in range(r,k)])
    return X

def construction_of_matrix(A,d,N1,N2):
    '''
    input   : dimension n, a matrix mxn A = [aij], a column vector d
              and two parameters N1, N2 with N1<N2

    output  : a solution of the system Ax=d
    '''
    
    n,m=A.dimensions()[1] , A.dimensions()[0] # we define the rows (m) and columns (n)    
    # construction of the matrix
    zerom1 = matrix(QQ,1,[0 for i in range(0, n)]).T;     # 0_{nx1}
    zerom2 = matrix(QQ,1,[0 for i in range(0, n)]+[N1]);  # [0_{1xn},N1]
    In = identity_matrix(ZZ, n);
    upper_block = block_matrix([[In,zerom1]]);
    medium_row = zerom2
    lower_block = block_matrix([[N2*A,-N2*d]]);
    final_matrix = block_matrix(3,[ [upper_block], [zerom2], [lower_block]])
    return final_matrix.T


def a_solution(n,a,b,Y):    
    '''
    input   : dimension n, a vector a = [a1,a2,...,an], a target integer b 
          and a parameter Y which represents a bound for the solution, i.e. 
          we search for solutions |x_i|<Y. So choose Y large.
          
    output  : a solution of the equation a1x1+a2x2+...+anxn=b, possibly satisfies |xi|<Y
    '''
    r=0;    
    #construction of the matrix
    zerom=matrix(QQ,1,[r for i in range(0, n)]); # part of the last row
    X=[Y for i in range(0, n)];
    L=matrix(QQ,2,[[0 for i in range(0, n)],[a[i] for i in range(0, n)]]);
    L=L.transpose();
    M = identity_matrix(ZZ, n);
    M=matrix(QQ,n,[1/X[i]*M.row(i) for i in range(0,n)]);
    Mb=block_matrix([[M,L]]);
    zerom1=block_matrix([[zerom,1,-b]]);  # this is the last row
    Mb1=Mb.stack(zerom1);    
    L=Mb1.LLL();   #use LLL algorithm for basis reduction
    L1=[[(L[j][i]-r)*X[i] for i in range(0,n)] for j in range(0,n+1)];#if the last row has 1/2 instead of 0
    L2=[vector(L1[i]).dot_product(vector(a))-b for i in range(0,n+1)];
    if L2.count(0)>0:
        print("Euclidean length of the solution:",float(vector(L1[L2.index(0)]).norm()))
        print("One solution is:",L1[L2.index(0)])
        return L1[L2.index(0)]
    else:
        print("NULL")
        return "NULL"


def a_solution_of_the_system(A,d,N1,N2):
    if A==_:
        return
    n = A.dimensions()[1] # was referred without being defined
    B=construction_of_matrix(A,d,N1,N2)
    Blll = B.LLL()
    #in order to find the solutions
    # you have to find the row that has the entry N1 i.e.
    # we are looking for a row of the form (....,N1,...)
    nrows = Blll.dimensions()[0]
    for i in range(nrows):
        if N1 in Blll.row(i):
            t = i
            solution = Blll.row(t)[:n]
            #print("solution:",solution)
            return solution
    print("error")
    return "error"


def babai(B,v):  # TODO: Replace with the one of Fpylll
    Y=[]
    i=0
    j=0
    row=int(B.nrows())
    col=int(B.ncols())
    w=vector([0 for i in range(0,len(v))])
    Gram=B.gram_schmidt()[0] # Gram_schmidt is a function of Sagemath
    w=vector(v)
    for j in range(row):        
        i=row-j-1
        c1=w.dot_product((Gram[i])) # dot product is a function of Sagemath
        c2=Gram[i].dot_product(Gram[i])
        l=c1/c2
        e=floor(l+0.5)*(B.row(i))
        Y.append(e)
        w=w-(l-floor(l+0.5))*Gram[i]-(floor(l+0.5)*B.row(i))   
    u=sum(Y)        
    return u



In [3]:
#///// Generation of the matrix and definition of the parameters #////  
    
def domains(index:int, R:int, R_bits:list):
    '''
    Constructs dynamically S_i according to predefined R-bits map (list), and the index selection
    Input
    -----
    index  : an index that picks a list from R_bits
    R      : bits
    R_bits : is a list of lists of the form [a,b], a,b integers with a<=b
    
    e.g.
    sage:R_bits = [[1,2], [2,4], [2,4,8], [1,2,4,8], [1,2,4,8,16]]
    sage:domains(2,10,R_bits)
    [(16, 31), (2, 3)] --> I_{R/2} x I_{R/4}
    
    sage:domains(1,10,R_bits)
    [(512, 1023), (16, 31)] --> I_R x I_{R/2}
    '''
    if index<1 or index>len(R_bits):
        return "index error"
    bounds = lambda R, den: (2 ** (R // den - 1), 2 ** (R // den) - 1)
    bisection = lambda R, dens: [bounds(R, d) for d in dens]
    bi = bisection(R, R_bits[index - 1]) # 0->1
    return bi 



def domain_selector(sector, n):
    '''
    This function is used to compute a random solution x of the system Ax=b where
    x in S. x has dimension n.
    Input:
    index   : is a positive integer
    sector  : is a list of lists [ [a1,a2],[b1,b2],...]
    n       : is a positive integer such that n = 0 mod len(sector)
    
    Output:
    returns a vector (w1,w2,...) where a1<=w1<=a2,....
    '''   
    if int(n)%int(len(sector)) !=0:
        print("n=",n," must be divisible by", len(sector))
        return 1,_,_
    a,_,_=check_indexes(n, R, index, mode, R_bits)
    if a==1:
        return 1,_,_
    #print(n,len(sector),n//len(sector),n/len(sector))
    rand = lambda n, bounds: [ZZ.random_element(bounds[0], bounds[1] + 1) for _ in range(n)]
    x = []
    for bounds in sector:
        x = x + rand(n//len(sector), bounds)
    return vector(x)

    
def gen_A_and_x(R, n, m, index, mode, R_bits):
    '''
    R : bits
    n : number of rows
    m : number of equations
    index = 1,2,3,4,5 or 6
    mode: denominator of \frac{R}{mode} that define the space I_{R/mode}, whose elements generate the matrix A.
    '''    
    a,_,_=check_indexes(n, R, index, mode, R_bits)
    if a==1:
        return 1,_,_
    bits = lambda R, mode: (2 ** (R / mode - 1), 2 ** (R /mode) - 1)

    left, right = bits(R, mode) # To generate the matrix A

    # Now, we generate a matrix with (n-1) rows, and random integer elements from the interval [left, right]
    A = matrix([vector([ZZ.random_element(left, right + 1) for _ in range(n)]) for _ in range(m)])
    # Now, we choose a solution with constraints
    dom_sel = domain_selector(domains(index, R, R_bits), n)
    if dom_sel[0] == 1:
        return 1,_,_
    else:
        x = domain_selector(domains(index, R, R_bits), n)
    # and the constant vector of the system
    b = (A * x)  
    #print("b=",b)
    return A, x, b


#\\\\ the attack \\\\#

# we choose the target vector
def target(index:int, R:int, n:int, R_bits:list):
    '''
    Defines the target vector according to the selected S_{index}
    n : the dimension of the target vector
    '''
    def unpack_lists_tuples2(list:list)->list:
        return [i for sub in list for i in sub]
    
    if index<1 or index>len(R_bits):
        return "index error"
    t_r = lambda R, den: 2 ** (R // den - 1) + 2 ** (R // den - 2)
    bisection_tr = lambda R, dens, n: unpack_lists_tuples2([[t_r(R, d)] * (n // len(dens)) for d in dens])
    target_vector = vector(bisection_tr(R, R_bits[index - 1], n))
    return target_vector 

def run_babai(index, R, A, R_bits):
    basis = kernel(A)
    if rank_of_L(A)[0] > 1:  # we want at least two rows to apply LLL reduction to matrix A
        B_lll = basis.LLL()  # before we execute Babai we reduce the basis with LLL or BKZ
    else:
        B_lll = basis
    # Alll=A.BKZ(block_size=35) # uncomment if you want to use BKZ
    n = A.ncols()
    t = target(index, R, n,  R_bits)
    # print(f'Target: {t}')

    # We execute the attack
    # First we execute approximate CVP using Babai's algorithm
    return babai(B_lll, t)

def Xd_list(alpha,babai_vector,sol,n): #Edit: just add n as argument
    '''
    input  : an integer alpha, usually 10 and the output of babai(.,.)
             sol is a solution of the non homogeneouos
             n is the dimension (number of columns of A)
    output : a list that contains the vectors, sol +j*babai_vector, for j=-alpha,...,alpha 
    '''
    Xdn = [] # Xd negative
    Xdp = [] # Xd positive
    Xd  = [] # the union of Xdn and Xdp
    if babai_vector!=vector([0]*n):
        Xdn = [vector(sol) - j*vector(babai_vector) for j in range(alpha)] 
        Xdp = [vector(sol) + j*vector(babai_vector) for j in range(alpha)] 
        Xd = Xdn+Xdp
    else: # i.e. in the case where babai provides the zero vector
        Xd = vector(sol)
        alpha = 0
    return Xd,alpha

#Modified functions for dynamic execution
def number_of_good_entries_single(list, bound):  # the number of entries in the interval [lower,upper]
    '''
    sage:number_of_good_entries_single([2,10,11,90],[90,100])
         (1,3)
    '''
    C = 0
    inbound = lambda x, low, up: (x <= up and x >= low)
    for i in range(len(list)):
        if inbound(list[i], *bound):
            C = C + 1
    return C, len(list) - C  # number of good and bad entries

def number_of_good_entries_multiple(list:list, bound:list):
    '''
    Dynamic definition of previous function number_of_good_entries
    :param list: [l1,l2,...,lk]
    :param bound: List of tuples with bounds [(lower1, upper1), ..., (lower-nth, upper-nth)]
    :return: after partioning list into n-packs, we check (lower[i],upper[i]) in the i-th pack
    
    Examples
    
    sage:number_of_good_entries_multiple([2,100,200,900],[(1,5),(900,1000)])
         (2,2)
    '''
    C = 0
    inbound = lambda x, low, up: (x <= up and x >= low) # boolean function
    inbound_couples = lambda x, bounds_list: sum([inbound(x, *bounds) for bounds in bounds_list]) > 0 #expensive(but easy to be faster)
    for i in range(len(list)):
        if inbound_couples(list[i], bound):
            C = C + 1
    return C, len(list) - C  # number of good and bad entries

def solution_verification_constraints(alpha:int, index:int, Xd:list, R:int, R_bits:list):
    #number_of_good_entries = number_of_good_entries_single if index==1 else number_of_good_entries_multiple
    number_of_good_entries = number_of_good_entries_multiple
    sector = domains(index, R, R_bits)
    # we want to check if our attack found a vector in S1 or S2
    M = []
    if alpha != 0:
        for i in range(2 * alpha):
            temp = number_of_good_entries(list(Xd[i]), sector)
            C = temp[0]
            M.append(C)
    else:
        temp = number_of_good_entries(list(Xd), sector)
        C = temp[0]
        return C
    return max(M)

#! this is a new function !#
def solution_verification(alpha, index, Xd, R, R_bits):
    number_of_good_entries = number_of_good_entries_multiple
    M = []
    sector = domains(index, R, R_bits)
    if alpha != 0:
        for i in range(2 * alpha):
            temp = number_of_good_entries(list(Xd[i]), sector)
            C = temp[0]
            M.append(C)
    else:
        temp = number_of_good_entries(list(Xd), sector)
        C = temp[0]
        M = C
    t = M.index(max(M))
    x = Xd[t]
    return x

def attack(A, sol, index, R, R_bits, alpha):
    # We execute the attack
    # First we execute approximate CVP using Babai's algorithm
    M = [] # will keep the solutions in this set
    n = A.ncols()
    m = A.nrows()
    babai_vector = run_babai(index, R, A, R_bits)
    Xd, alpha = Xd_list(alpha, babai_vector, sol,n)
    out = solution_verification_constraints(alpha, index, Xd, R, R_bits)
    #z = solution(alpha, index, Xd, R, R_bits)
    #print(z[0:n])
    #print(A*matrix(z[0:n]).T)
    return out,Xd

def stats(mode, n, m, count, H):
    import numpy as np
    print("bits of A=(aij)=", "R /", mode)
    print("we picked R_bits to be:",R_bits[index-1])
    print("unknowns=", n)
    print("equations=", m)
    print("number of instances considered:", count)
    print("average number of good hits:", "{:.2f}".format(round(np.mean(H), 2)), ",", "percentage :",
          float(100 * np.mean(H) / n))
    print("the minumum of succeded hits:", min(H))
    print("the maximum :", max(H))
    if max(H) == n:
        print("+++ We found at least one solution +++")
        
class Model:
    def __init__(self, index, R, n, m, alpha, mode, R_bits):
        self.index = index
        # definition of I_R
        self.R = R  # choose for simplicity R to be even
        # definition of n and m
        self.n = n  # so we work in dimension n,i.e. we have n unknowns
        self.m = m  # we have m equations.
        self.alpha = alpha # heuristic integer
        self.mode = mode # denotes the denominator of fraction R/mode
        self.R_bits = R_bits
        
def payload_exec(m:Model):
    A, x, b = gen_A_and_x(m.R, m.n, m.m, m.index, m.mode, m.R_bits)
    if A==1:
        return -1,_,_,_
    N1, N2 = int(2 ** m.R), int(2 ** (m.R + 2))
    sol = a_solution_of_the_system(A, matrix(b).T, N1, N2)  # Non-Homogeneous system solution
    hits,Xd = attack(A, sol, m.index, m.R, m.R_bits, m.alpha)
    S = solution_verification(m.alpha, m.index, Xd, m.R, m.R_bits)
    return hits,S,b,A

### Linear execution
def run(model:Model,count:int):
    H=[]
    S=[]
    bvector=[]
    Alist=[]
    for i in range(count):
        P=payload_exec(model)
        if P[0]==-1:
            return
        H.append(P[0]) # I am not sure that this is memory efficient
        # uncomment the follow lines if you want to verify your solutions
        #S.append(P[1])
        #bvector.append(P[2])
        #Alist.append(P[3])
    stats(model.mode, model.n, model.m, count, H)
    #[print("verification of", j," the solution:",Alist[j]*matrix(S[j]).T==matrix(bvector[j]).T) for j in range(count)]

In [4]:
R_bits = [[1], [1,2], [2,4], [2,4,8], [1,2,4,8], [1,2,4,8,16],[1,3,9,27],[1,2,3,4,8]]
R,n,m,index,mode = 40, 40, 4, 2, 8
inputs = Model(index=index, R=R, n=n, m=m, alpha=10, mode=8, R_bits=R_bits)
run(inputs, 2)

bits of A=(aij)= R / 8
we picked R_bits to be: [1, 2]
unknowns= 40
equations= 4
number of instances considered: 2
average number of good hits: 20.00 , percentage : 50.0
the minumum of succeded hits: 20
the maximum : 20


In [5]:
R,n,m,index,mode = 48, 40, 4, 1, 8
inputs = Model(index=index, R=R, n=n, m=m, alpha=10, mode=8, R_bits=R_bits)
run(inputs, 1)

bits of A=(aij)= R / 8
we picked R_bits: [1]
unknowns= 40
equations= 4
number of instances considered: 1
average number of good hits: 40.00 , percentage : 100.0
the minumum of succeded hits: 40
the maximum : 40
+++ We found at least one solution +++
